# 用SVD实现ridge regression

今天在实现岭回归的时候发现了一个问题，我写好了如下的岭回归代码，接着我将正则化系数设为0（退化为线性回归），输出拟合曲线的系数，然后我又将其与线性回归的系数输出进行比较，发现两者不一致且相差很大。

In [6]:
import numpy as np
import matplotlib.pyplot as plt
def polyfit(x, t, M):
    '''polynomial curve fitting
    # Arguments:
        x: vector of input variables
        t: targets of input variables
        M: degree of polynomial model
    # Returns:
        coefficients of the polynomial model
    '''
    X = np.array([[xx ** m for m in range(M+1)] for xx in x], dtype='float32')
    #return np.linalg.pinv(X).dot(t)
    # more accurate version, equivalent to pinv
    return np.linalg.lstsq(X, t)[0]

M = 9
_lambda = 0
x = np.linspace(0, 1, 501)
t = np.sin(2 * np.pi * x)
# 生成0~1之间等间距的10个点
n = 10
x_tr = np.linspace(0, 1, n)
t_tr = np.sin(2 * np.pi * x_tr) + np.random.normal(0, 0.3, n)
Phi = np.array([[xx ** m for m in range(M+1)] for xx in x_tr], dtype='float32')    
w_ridge = np.linalg.solve(Phi.T.dot(Phi)+_lambda * np.eye(M+1), Phi.T.dot(t_tr))
wgt_info = ''
print 'weights of ridge regression(lambda={}):'.format(_lambda)
for i, ww in enumerate(w_ridge):
    wgt_info += 'w_{}={:.2f}, '.format(i, ww)
print wgt_info

w_lr = polyfit(x_tr, t_tr, M)
wgt_info = ''
print 'weights of linear regression:'
for i, ww in enumerate(w_lr):
    wgt_info += 'w_{}={:.2f}, '.format(i, ww)
print wgt_info

weights of ridge regression(lambda=0):
w_0=-0.17, w_1=19.90, w_2=-141.49, w_3=485.66, w_4=-503.59, w_5=-1146.10, w_6=2537.15, w_7=-79.13, w_8=-2483.43, w_9=1311.17, 
weights of linear regression:
w_0=-0.24, w_1=73.92, w_2=-1227.70, w_3=9413.51, w_4=-40494.25, w_5=106457.07, w_6=-175722.05, w_7=177370.64, w_8=-99533.63, w_9=23662.71, 


对于这个问题我百思不得其解，于是求助于网络，经过一番查找，大概得到的答案是直接用solve函数求解在式子左边的矩阵接近奇异时会出现数值上的不稳定，解决方法是用奇异值分解过滤掉接近0的奇异值。

回顾岭回归的求解公式：
$$w^*=(X^\top X+\lambda I)^{-1}X^\top y$$
将矩阵$X$进行奇异值分解有：
$$ X=U\Sigma V^\top$$
我们要求解的是伪逆矩阵$X^\dagger=(X^\top X+\lambda I)^{-1}X^\top$，将分解式代入有：
$$\begin{aligned}(X^\top X+\lambda I)^{-1}X^\top &=(V\Sigma (U^\top U)\Sigma V^\top+\lambda I)^{-1}\cdot V\Sigma U^\top\\&=(V\Sigma^2 V^T+\lambda V V^\top)^{-1}\cdot V\Sigma U^\top\\&=\big(V(\Sigma^2 +\lambda I) V^\top\big)^{-1}\cdot V\Sigma U^\top\\&=(V^\top)^{-1}\cdot(\Sigma^2+\lambda I)^{-1}\cdot(V)^{-1}\cdot V\Sigma U^\top\\&=V(\Sigma^2+\lambda I)^{-1}\Sigma U^\top\\&=V \cdot diag(\frac{\sigma_1}{\sigma_1^2+\lambda},...,\frac{\sigma_k}{\sigma_k^2+\lambda})\cdot U^\top\end{aligned}$$
实现：

In [8]:
def ridge_regression(x, t,_lambda, M):
    '''ridge regression
    # Arguments:
        x: vector of input variables
        t: targets of input variables
        M: degree of polynomial model
    # Returns:
        coefficients of the polynomial model    
    '''
    Phi = np.array([[xx ** m for m in range(M+1)] for xx in x], dtype='float32')
    U, S, Vh = np.linalg.svd(Phi, full_matrices=False)
    Ut = U.T.dot(t)
    #return np.linalg.solve(Phi.T.dot(Phi) + _lambda * np.eye(M+1)), Phi.T.dot(t))
    return reduce(np.dot, [Vh.T, np.diag(S/(S**2 + _lambda)), Ut])

测试：

In [9]:
M = 9
_lambda = 0
w_ridge = ridge_regression(x_tr, t_tr, _lambda, M)
wgt_info = ''
print 'weights of ridge regression(lambda={}):'.format(_lambda)
for i, ww in enumerate(w_ridge):
    wgt_info += 'w_{}={:.2f}, '.format(i, ww)
print wgt_info

w_lr = polyfit(x_tr, t_tr, M)
wgt_info = ''
print 'weights of linear regression:'
for i, ww in enumerate(w_lr):
    wgt_info += 'w_{}={:.2f}, '.format(i, ww)
print wgt_info

weights of ridge regression(lambda=0):
w_0=-0.24, w_1=73.92, w_2=-1227.70, w_3=9413.51, w_4=-40494.27, w_5=106457.13, w_6=-175722.14, w_7=177370.73, w_8=-99533.68, w_9=23662.72, 
weights of linear regression:
w_0=-0.24, w_1=73.92, w_2=-1227.70, w_3=9413.51, w_4=-40494.25, w_5=106457.07, w_6=-175722.05, w_7=177370.64, w_8=-99533.63, w_9=23662.71, 


可以看到，当正则系数等于0时，svd版本的岭回归的结果和线性回归的结果基本一致。